## ws_sp_toolbox.ipynb
### run experiments with SP's toolbox
### WESmith 05/07/20

In [ ]:
import networkx as nx
from networkx.drawing.nx_pydot import pydot_layout
import toolbox as tb
import pyphi
import numpy as np
import pandas as pd

In [ ]:
def get_net(ledges, nodes, funcs, title=''):
    # wrapper for SP tools
    i, j = zip(*ledges)
    indexLUT = dict([(l,ord(l)-ord(nodes[0])) for l in sorted(set(i+j))])
    edges = [(indexLUT[i],indexLUT[j]) for i,j in ledges]
    net = tb.Net(edges=edges, title=title)
    for j,k in zip(nodes, funcs):
        net.get_node(j).func = k
    return net

In [ ]:
def get_phi(perm):
    try:
        return net.phi(perm)
    except:
        return -1 # state isn't reachable

In [ ]:
# example used in Mayner documents
edges =  [('A', 'B'), ('A', 'C'), 
          ('B', 'A'), ('B', 'C'), 
          ('C', 'A'), ('C', 'B')]
nodes = ['A', 'B', 'C']
funcs = [tb.or_func, tb.and_func, tb.xor_func]

In [ ]:
net = get_net(edges, nodes, funcs)

In [ ]:
net.draw()

In [ ]:
print(nx.info(net.graph))

In [ ]:
for k in nodes:
    print(net.get_node(k))

In [ ]:
df=net.tpm
df

- above table agrees with slide 39 of pcbi.1006343.s001.pdf

---

In [ ]:
print(f"Probability distributions: {dict([(l,net.node_pd(net.get_node(l))) for l in nodes])}")

- interpretation: from above table, A is on 75% of the time, B is on 25% of the time, C is on 50% of the time
- this consistent with output

---

In [ ]:
for k in nodes:
    print('{1}, {0}'.format(net.get_node(k), net.node_state_counts(net.get_node(k))))

- above are truth tables for each node: how many times 0 shows, how many times 1 shows
- eg: first one is 'or': 0 shows once, 1 shows three times

---

In [ ]:
state = (0,1,1)
out = [net.eval_node(net.get_node(k), state) for k in nodes]
out

- this evaluates the output at t+1 for the node when system state is as given at t
- above example reproduces one row of the TPM

---

In [ ]:
net.phi('100')

- this phi agrees with phi from slide 227 of above-referenced pdf

---

In [ ]:
# make histogram of phi over alls states
# need helper function because of try:
dd = [(k, get_phi(k)) for k in df.index]

In [ ]:
dd

In [ ]:
dff = pd.DataFrame(dd)
ax = dff.plot.hist(bins=100)

---

### now run two identical 3-node systems, bidirectionally coupled at a node